# Example of parsing a .dec decay file with Lark

This file demonstrates how to parse a decfile with only the Lark grammar definition. Much of the functionality demonstrated here is already provided as part of decaylanguage in a more thorough and detailed form. This example only serves to show the key data structures used.

In [ ]:
from __future__ import annotations

import re

from lark import Lark, Tree

from decaylanguage import data

In [ ]:
with data.basepath.joinpath("decfile.lark").open() as f:
    grammar = f.read()

with open("../tests/data/test_example_Dst.dec") as f:
    dec_file = f.read()

For illustration - the grammar Lark file:

In [ ]:
print(grammar)

For illustration - the .dec decay file:

In [ ]:
print(dec_file)

Define a helper function to dynamically load the model names needed to parse the decfile

In [ ]:
def edit_model_name_terminals(t) -> None:
    """
    Edits the terminals of the grammar to replace the model name placeholder with the actual names of the models.
    """
    decay_models = ("VSS", "VSP_PWAVE", "PHSP", "PI0_DALITZ")
    modelstr = rf"(?:{'|'.join(re.escape(dm) for dm in sorted(decay_models, key=len, reverse=True))})"
    if t.name == "MODEL_NAME":
        t.pattern.value = t.pattern.value.replace("MODEL_NAME_PLACEHOLDER", modelstr)

Parse the .dec decay file.

In [ ]:
l = Lark(grammar, parser="lalr", lexer="auto", edit_terminals=edit_model_name_terminals)
parsed_dec_file = l.parse(dec_file)

In [ ]:
def number_of_decays(parsed_file):
    """Returns the number of particle decays defined in the parsed .dec file."""
    return len(list(parsed_file.find_data("decay")))


print("# of decays in file =", number_of_decays(parsed_dec_file))

In [ ]:
def list_of_decay_trees(parsed_file):
    """Return a list of the actual decays defined in the .dec file."""
    return list(parsed_file.find_data("decay"))

In [ ]:
def get_decay_mode_details(decay_mode_Tree):
    """Parse a decay mode tree and return the relevant bits of information in it."""
    bf = (
        next(iter(decay_mode_Tree.find_data("value"))).children[0].value
        if len(list(decay_mode_Tree.find_data("value"))) == 1
        else None
    )
    bf = float(bf)
    products = tuple(
        [
            p.children[0].value
            for p in decay_mode_Tree.children
            if isinstance(p, Tree) and p.data == "particle"
        ]
    )
    model = (
        next(iter(decay_mode_Tree.find_data("model"))).children[0].value
        if len(list(decay_mode_Tree.find_data("model"))) == 1
        else None
    )
    return (bf, products, model)

Finally, digest all Lark's Tree objects parsed and collect the information of all defined decays.

In [ ]:
decays = {}

for tree in list_of_decay_trees(parsed_dec_file):
    if tree.data == "decay":
        if tree.children[0].children[0].value in decays:
            print(
                f"Decays of particle {tree.children[0].children[0].value} are redefined! Please check your .dec file."
            )
        decays[tree.children[0].children[0].value] = []
        for decay_mode in tree.find_data("decayline"):
            decays[tree.children[0].children[0].value].append(
                get_decay_mode_details(decay_mode)
            )

For illustration - print out the decay modes:

In [ ]:
def print_decay(dec, final_state):
    """Pretty print of the decay modes of a given particle."""
    print(dec)
    for fs in final_state:
        print(f"{fs[0]:12g} : {'  '.join(p for p in fs[1]):50s} {fs[2]:15s}")

In [ ]:
print_decay("pi0", decays["pi0"])

In [ ]:
for particle, decay_info in decays.items():
    print_decay(particle, decay_info)

For illustration - produce a dot plot of a decay Tree:

In [ ]:
from IPython.display import Image
from lark.tree import pydot__tree_to_png  # requires pydot

pydot__tree_to_png(
    list_of_decay_trees(parsed_dec_file)[0], filename="decay.png", rankdir="LR"
)

Image(filename="decay.png")